**Compute DEGs and pathway enrichments**


In [1]:
library(edgeR)
library(sva)
library(fgsea)
library(tidyr)
source('../ABCA7lof2//degs.r')
set.seed(5)

Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel



#### Compute gene scores (DEGs)

In [1]:
all_data = readRDS('./processed_data/single_cell/stats_input_data_0825.rds')

In [3]:
summed_counts = all_data$summed_counts_by_ind # does it make sense to be summing on the counts vs taking the mean of the counts? --> compare to nebula? --> ok because we are normalizing?
meta = all_data$summary
meta$seq_batch = ifelse(meta$seq_batch=='JBM',1,0)
expressed = all_data$expressed10

In [4]:
# filter out individuals with too few cells per celltype
keep = summed_counts$ncells>=10
summed_counts_indexed = summed_counts$summed_counts[,keep]

In [5]:
# compute degs (all samples)
vars = c('LOF', 'amyloid', 'nft',  'msex', 'age_death', 'pmi', 'seq_batch', 'APOE4')
limma_inputs = get_limma_inputs(summed_counts_indexed, expressed, meta, vars)
aggs = limma_inputs$aggs
metadata = limma_inputs$metadata


degs_all = lapply(names(aggs)[!names(aggs)%in%c('Vascular')], function(x) RunDiffExprAnalysisLimma(aggs[[x]], metadata[[x]], n.sv=NULL))
names(degs_all) = names(aggs)[!names(aggs)%in%c('Vascular')]

Number of significant surrogate variables is:  10 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  9 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  7 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  9 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  7 
Iteration (out of 5 ):1  2  3  4  5  

In [6]:
# get the ids
ids = strsplit(colnames(summed_counts_indexed), '[.]')
ids = unlist(lapply(1:length(ids), function(x) ids[[x]][2]))

In [7]:
# compute degs (no APOE4 samples)
vars = c('LOF', 'amyloid', 'nft',  'msex', 'age_death', 'pmi', 'seq_batch')
sele = rownames(meta)[meta$APOE4==0]
sc_noAPOE4 = summed_counts_indexed[,ids%in%sele]
limma_inputs = get_limma_inputs(sc_noAPOE4, expressed, meta[sele,], vars)
aggs = limma_inputs$aggs
metadata = limma_inputs$metadata
degs_noAPOE4 = lapply(names(aggs)[!names(aggs)%in%c('Vascular')], function(x) RunDiffExprAnalysisLimma(aggs[[x]], metadata[[x]], n.sv=NULL, exclude_apoe=TRUE))
names(degs_noAPOE4) = names(aggs)[!names(aggs)%in%c('Vascular')]


Number of significant surrogate variables is:  6 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  6 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  4 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  

In [8]:
# compute degs (no APOE4 samples)
vars = c('LOF', 'amyloid', 'nft',  'msex', 'age_death', 'pmi', 'APOE4')
sele = rownames(meta)[meta$seq_batch==1]
sc_JBM = summed_counts_indexed[,ids%in%sele]
limma_inputs = get_limma_inputs(sc_JBM, expressed, meta[sele,], vars)
aggs = limma_inputs$aggs
metadata = limma_inputs$metadata
degs_JBM = lapply(names(aggs)[!names(aggs)%in%c('Vascular')], function(x) RunDiffExprAnalysisLimma(aggs[[x]], metadata[[x]], n.sv=NULL, exclude_batch=TRUE))
names(degs_JBM) = names(aggs)[!names(aggs)%in%c('Vascular')]

Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  4 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  3 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  3 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  3 
Iteration (out of 5 ):1  2  3  4  5  

In [9]:
# exclude both
vars = c('LOF', 'amyloid', 'nft',  'msex', 'age_death', 'pmi')
sele = rownames(meta)[(meta$seq_batch==1) & (meta$APOE4==0)]
sc_exclude_both = summed_counts_indexed[,ids%in%sele]
limma_inputs = get_limma_inputs(sc_exclude_both, expressed, meta[sele,], vars)
aggs = limma_inputs$aggs
metadata = limma_inputs$metadata
degs_exclude_both = lapply(names(aggs)[!names(aggs)%in%c('Vascular')], function(x) RunDiffExprAnalysisLimma(aggs[[x]], metadata[[x]], n.sv=NULL, exclude_both=TRUE))
names(degs_exclude_both) = names(aggs)[!names(aggs)%in%c('Vascular')]

Number of significant surrogate variables is:  4 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  4 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  4 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  3 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

In [11]:
# save the results
degs = list()
degs[['degs_all']] = degs_all
degs[['degs_exclude_both']] = degs_exclude_both
degs[['degs_JBM']] = degs_JBM
degs[['degs_noAPOE4']] = degs_noAPOE4

saveRDS(degs, './processed_data/single_cell/pseudobulk_degs_0825.rds')

In [12]:
# save deg scores as matrix
out = list()
scores = get_deg_scores(degs$degs_all)
for(i in names(scores)){
    df = as.data.frame(scores[[i]]$scores)
    colnames(df) = c('score')
    df$gene = rownames(df)
    df$celltype = i
    out[[i]] = df
}
all_scores = as.data.frame(do.call('rbind', out)%>%pivot_wider(., values_from='score', names_from='celltype'))
rownames(all_scores) = all_scores$gene
all_scores$gene = NULL
all_scores[is.na(all_scores)] = 0
write.csv(all_scores, './processed_data/single_cell/all_scores_0825.csv')

#### Run pathway enrichments (FGSEA)

In [2]:
# load degs
degs = readRDS('./processed_data/single_cell/pseudobulk_degs_0825.rds')$degs_all
scores = get_deg_scores(degs)

# compute fgsea results
all_paths = read.csv('./processed_data/genesets/all_paths.csv', row.names = 'X')
pathways = as.list(as.data.frame(t(all_paths)))
temp = lapply(names(pathways), function(x) pathways[[x]][!(pathways[[x]]=='')])
names(temp) = names(pathways)
              
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%



In [41]:
# save the fgsea results
saveRDS(out, './processed_data/for_plotting/fgsea_out_0825.rds')

# save all fgsea results
df = as.data.frame(do.call('rbind', out))
df = df[,!colnames(df)=='leadingEdge']
write.csv(df, './processed_data/for_plotting/fgsea_out_0825.csv')

# save LE genes per celltype
for(i in names(out)){
    res = out[[i]]
    res = res[res$pval<0.05,]

    df = as.data.frame(unique(unname(unlist(res$leadingEdge))))
    colnames(df) = 'gene'
    write.csv(df, paste0(paste0('./processed_data/for_plotting/leading_edge_0825', i), '.csv'))
}

In [42]:
# save all LEs
out = readRDS('./processed_data/for_plotting/fgsea_out_0825.rds')
res = do.call('rbind', out)
res$score = sign(res$NES) * -log10(res$pval)
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]
# save all leading edge genes
df = as.data.frame(unique(unname(unlist(res$leadingEdge))))
colnames(df) = 'gene'
write.csv(df, './processed_data/for_plotting/leading_edge_0825.csv')

In [16]:
# save supp table
res = list()
for(i in names(out)){
    T = out[[i]]
    T = T[T$pval<0.05,]
    T = T[order(T$pval,decreasing=FALSE),]
    res[[i]] = T
}
temp = do.call('rbind', res)
temp2 = temp
x = unlist(lapply(temp$leadingEdge, function(x) paste(x, collapse=', ')))
temp$leadingEdge = x
write.csv(temp, './supplementary_tables/data_s3.csv')

#### Run FGSEA for specific genesets of interest

###### NFKB

In [47]:
# enrichment for nfkb genesets
pathways = readRDS('../ABCA7lof/processed_data/pathway_databases/pathways.rds')$all_paths
                       
o = get_gset_names_by_category(c('kappa'), names(pathways))
temp2 = pathways[o]
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp2, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)
          
# save kappa paths

saveRDS(out, './processed_data/for_plotting/fgsea_out_all_kappa.rds')

  |======================================================================| 100%



In [48]:
# save supp table
temp = do.call('rbind', out)
x = unlist(lapply(temp$leadingEdge, function(x) paste(x, collapse=', ')))
temp$leadingEdge = x
write.csv(temp, './supplementary_tables/data_s5.csv')

###### Lipid - general

In [49]:
# enrichment for lipid genesets
pathways = readRDS('../ABCA7lof/processed_data/pathway_databases/pathways.rds')$all_paths

o = get_gset_names_by_category(c('sterol', 'lipid', 'glycer', 'fatt', 'ceramide', 'phosphatidyl'), names(pathways))
temp2 = pathways[o]
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp2, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)
   
# save lipid paths
res = out$Ex
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]
saveRDS(res, './processed_data/for_plotting/fgsea_out_ex_lipid.rds')
             
# save lipid paths
saveRDS(out, './processed_data/for_plotting/fgsea_out_all_lipid.rds')

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%

  |======================================================================| 100%



In [50]:
# save supp table
temp = do.call('rbind', out)
x = unlist(lapply(temp$leadingEdge, function(x) paste(x, collapse=', ')))
temp$leadingEdge = x
write.csv(temp, './supplementary_tables/data_s4.csv')

###### Lipid synthesis

In [4]:
# enrichment for lipid genesets
degs = readRDS('../processed_data/single_cell/pseudobulk_degs_0825.rds')$degs_all
scores = get_deg_scores(degs)

pathways = readRDS('../../ABCA7lof/processed_data/pathway_databases/pathways.rds')$all_paths

o = get_gset_names_by_category(c('lipid biosynthesis', 'lipid storage', 'lipid synthesis', 'Fatty Acyl-CoA Biosynthesis', 'Triglyceride Biosynthesis'), names(pathways))
temp2 = pathways[o]
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp2, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)
   
# save lipid paths
res = out$Ex
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]

x = temp2[res$pathway]

out = c()
for(i in names(x)){
    t = as.data.frame(x[[i]])
    names(t) = 'value'
    t$grp = i
    out = rbind(out, t)
}

#write.csv(out, '../processed_data/common_var/lipid_synthesis_gsets.csv')

In [6]:
head(res)

x,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
Ex,Fatty Acyl-CoA Biosynthesis Homo sapiens R-HSA-75105,0.01920904,0.05146695,0.15627915,-0.5188635,-1.587481,26,"ACLY, EL...."
Ex,negative regulation of lipid storage (GO:0010888),0.02198835,0.05146695,0.13288187,0.7074500,1.625526,10,"ITGAV, N...."
Ex,SREBP control of lipid synthesis Homo sapiens h s1pPathway,0.02205726,0.05146695,0.14107290,-0.7562360,-1.615867,7,"LDLR, SC...."
Ex,Triglyceride Biosynthesis Homo sapiens R-HSA-75109,0.04827425,0.08447994,0.09935352,-0.4249140,-1.424228,39,"AGPAT1, ...."


###### Lipid oxidation

In [23]:
# enrichment for lipid genesets
degs = readRDS('../processed_data/single_cell/pseudobulk_degs_0825.rds')$degs_all
scores = get_deg_scores(degs)

pathways = readRDS('../../ABCA7lof/processed_data/pathway_databases/pathways.rds')$all_paths

o = get_gset_names_by_category(c('oxidation', 'TCA', 'lipolysis'), names(pathways))
temp2 = pathways[o]
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp2, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)
   
# save lipid paths
res = out$Ex
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]

x = temp2[res$pathway]

out = c()
for(i in names(x)){
    t = as.data.frame(x[[i]])
    names(t) = 'value'
    t$grp = i
    out = rbind(out, t)
}

write.csv(out, '../processed_data/common_var/lipid_ox_gset.csv')

  |======================================================================| 100%



In [25]:
head(res)

x,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
Ex,The citric acid (TCA) cycle and respiratory electron transport Homo sapiens R-HSA-1428517,7.676197e-05,0.002149335,0.5384341,0.4487719,1.771416,125,"PDHX, ND...."
Ex,fatty acid beta-oxidation using acyl-CoA oxidase (GO:0033540),5.095301e-03,0.071334214,0.2798657,0.7540411,1.746784,10,"SCP2, AM...."
Ex,Beta-oxidation of pristanoyl-CoA Homo sapiens R-HSA-389887,3.762636e-02,0.350315194,0.1008600,0.7002942,1.527255,8,"SCP2, AM...."


###### ROS

In [26]:
# enrichment for lipid genesets
degs = readRDS('../processed_data/single_cell/pseudobulk_degs_0825.rds')$degs_all
scores = get_deg_scores(degs)

pathways = readRDS('../../ABCA7lof/processed_data/pathway_databases/pathways.rds')$all_paths

o = get_gset_names_by_category(c('oxidative', 'reactive'), names(pathways))
temp2 = pathways[o]
out = lapply(names(scores), function(x) cbind(x, fgsea(pathways = temp2, stats = scores[[x]]$scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)))
names(out) = names(scores)
   
# save lipid paths
res = out$Ex
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]


x = temp2[res$pathway]

out = c()
for(i in names(x)){
    t = as.data.frame(x[[i]])
    names(t) = 'value'
    t$grp = i
    out = rbind(out, t)
}

write.csv(out, '../processed_data/common_var/ros_gset.csv')

  |======================================================================| 100%



In [27]:
head(res)

x,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
Ex,negative regulation of response to reactive oxygen species (GO:1901032),0.005287009,0.1063444,0.2747175,0.8493056,1.699412,6,"PDE8A, L...."
Ex,response to reactive oxygen species (GO:0000302),0.018268048,0.1063444,0.1410729,0.4810188,1.543864,38,"PDCD10, ...."
Ex,positive regulation of reactive oxygen species metabolic process (GO:2000379),0.018936635,0.1063444,0.1410729,0.6744051,1.637089,12,"GSTP1, M...."
Ex,negative regulation of oxidative stress-induced neuron death (GO:1903204),0.022765441,0.1063444,0.1300861,0.7353675,1.592112,8,"NONO, PA...."
Ex,regulation of oxidative stress-induced neuron intrinsic apoptotic signaling pathway (GO:1903376),0.024169184,0.1063444,0.1269498,0.7805687,1.561873,6,"NONO, PA...."


#### Save some additional data

In [15]:
# save deg supp table
degs = readRDS('./processed_data/single_cell/pseudobulk_degs_0825.rds')$degs_all
temp = lapply(names(degs), function(x) cbind(rownames(degs[[x]]$res), x, degs[[x]]$res))
temp = do.call('rbind', temp)
colnames(temp) = c('gene', 'cell_type', 'logFC', 'AveExpr', 't', 'P.Value', 'adj.P.Val', 'B')
rownames(temp) = NULL
write.csv(temp, './supplementary_tables/data_s9.csv')

In [1]:
out = readRDS('./processed_data/for_plotting/fgsea_out_0825.rds')
res = do.call('rbind', out)
res$score = sign(res$NES) * -log10(res$pval)
res = res[order(res$pval,decreasing=FALSE),]
res = res[res$pval<0.05,]

In [4]:
temp = read.csv('./processed_data/for_plotting/leading_edge_0825Ex.csv')